# Partie 2: Réseaux de neurones

Nous allons ici une seconde méthode d'indexation, l'indexation par réseau de neurones.

> Le code utilisé provient du tutoriel de la libairie Keras créée par François Chollet. https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html .
Afin de visualiser les changements dans le code, le code changé est en commentaire et les explications sont représentées par ###### explications ######

---
## 2.1. Retour sur l'architecture

Nous utiliserons dans cette partie l'architecture vue dans la première partie. En particulier les dossiers train (jeu de données d'apprentissage), le dossier validation et le dossier test.

---
## 2.2. L'apprentissage

Comme expliqué plus tôt, la première étape d'un réseau de neurones est l'apprentissage.
Lors de cette étape nous allons construire le modèle qui contiendra la structure globale du réseau de neurones. Ce modèle est ensuite enregistré au format HDF5 (.h5). Ce fichier HDF5 sera utilisé par le programme test.py pour prédire les classes des images qui se trouvent dans l’ensemble de test.

---
### 2.2.1. Les imports

In [1]:
from keras.preprocessing.image import ImageDataGenerator, image
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.utils import plot_model
from sklearn.metrics import classification_report, confusion_matrix

import numpy as np
import shutil
import os
import matplotlib.pyplot as plt

Using TensorFlow backend.


---
### 2.2.2. Le redimensionnement des images

Toutes images contenues dans la database n'ont pas la même taille. Il faut donc commencer par les redimensionner uniformément. En prenant la plus petite taille d'image on s'assure d'en garder une qualité correcte.

On n'oublie pas de donner les chemins d'accès aux images. <span style='color:red'>(n'oubliez pas de préciser vos chemins vers votre base si votre architecture de dossiers est différente)</span>
<br><br>
Pour définir la bonne valeur d'epoch, il faut procéder à quelques essais afin d'éviter le sur-apprentissage.
<br><br>
Pour définir le batch_size, il faut faire un compromis entre performance et vitesse. En effet, plus la taille du batch est élevée plus le modèle apprendra rapidement mais en étant moins performant.

Selon la base d'image que vous possédez, ces paramètres doivent être modifié. Dans notre cas, ces valeurs offraient un bon compromis entre l'apprentissage et la performance du modèle. 

In [4]:
# dimensions of our images.
#img_width, img_height = 150, 150
img_width, img_height = 64, 64

train_data_dir = 'database/train'
validation_data_dir = 'database/validation'

##### On ajoute un paramètre et on automatise les autres #####
#nb_train_samples = 2000
#nb_validation_samples = 800
nb_classes = sum([len(d) for r, d, files in os.walk(train_data_dir)])
nb_train_samples = sum([len(files) for r, d, files in os.walk(train_data_dir)])
nb_validation_samples = sum([len(files) for r, d, files in os.walk(validation_data_dir)])

epochs = 25
batch_size = 8

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

### 2.2.3. Les couches de neurones
Le modèle est séquentiel et composé de 4 couches. Chaque couche comporte plus de neurones que la précédente pour affiner la précision de la classification. 

Nous avons aussi ajouté un dropout sur l'avant dernière couche. Il permet de réduire le sur-apprentissage en empêchant deux couches d'avoir le même schéma. 

In [5]:
#### Le modèle est séquentiel et composé de 4 couches ####
model = Sequential()
#model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Conv2D(16, (5, 5), input_shape=input_shape, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(32, (3, 3)))
model.add(Conv2D(32, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#model.add(Conv2D(64, (3, 3)))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

####### ajout d'une couche #######
model.add(Conv2D(128, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
#model.add(Dense(64))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dropout(0.25))
#model.add(Dense(1))
model.add(Dense(nb_classes))
#model.add(Activation('sigmoid'))
model.add(Activation('softmax'))

#model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_8/MaxPool' (op: 'MaxPool') with input shapes: [?,1,1,128].

### 2.2.4. Augmentation des données

Comme nous travaillons sur un petit jeu de test ( il faudra avoir des milliers d'images pour que le réseau de neurones soit complétement efficace), nous allons utiliser la **data augmentation**. Ce processus permet d'augmenter le nombre de données artificiellement en effectuant des transformations (rotation, zooms, translations, etc) sur des images existantes.

In [9]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

'''
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
'''
train_generator = train_datagen.flow_from_directory(
    train_data_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size, 
    classes=None, class_mode='categorical',
    color_mode='rgb',
    interpolation='bilinear')

'''
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
'''
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size, classes=None, 
    class_mode='categorical',
    color_mode='rgb', interpolation='bilinear')

####
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=2)

#model.save_weights('first_try.h5')

# sauvegarde du modèle 
model.save('CorelDB_model.h5')

#### affichage statistiques #####
#plot_model(model, to_file="model_v1.png")

#plt.figure()
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
#plt.title("Précision du modèle")
#plt.ylabel("Précision")
#plt.xlabel("Epoch")
#plt.legend(["Apprentissage", "Validation"], loc="upper left")
#plt.savefig('precision.png')
#plt.show(block='false')

Found 272 images belonging to 4 classes.
Found 0 images belonging to 0 classes.


RuntimeError: You must compile your model before using it.

---
## 2.3. L'indexation

Pour indexer de nouvelles images nous allons charger le modèle créé précédement. Nous lui spécifions aussi les chemins des répertoires d'images à traiter. 

In [10]:
def reports(y_pred, y_true, classs_names):
    classification = classification_report(y_true, y_pred, target_names=classs_names)
    confusion = confusion_matrix(y_true, y_pred)

    return classification, confusion

#chargement du modele
model = load_model('CorelDB_model.h5')

#taille des images
img_width, img_height = 64, 64

#repertoire à tester et de résultats
img_dir = "database/test"
resu_dir = "database/result"

#nb images contenues dans le répertoire de test
nb_test_samples = sum([len(files) for r, d, files in os.walk(img_dir)])

#on récupère les classes des images test et on les ordonne pour éviter des erreurs de stockage
class_names = next(os.walk(img_dir))[1]
class_names.sort()

#on récupère les images et on les confronte au modèle
batch_holder = np.zeros((nb_test_samples, img_width, img_height, 3))
y_true = np.zeros(nb_test_samples)

i = 0
for dirpath, dirnames, filenames in os.walk(img_dir):
    for imgnm in filenames:
        img = image.load_img(os.path.join(dirpath, imgnm), target_size=(img_width, img_height))
        batch_holder[i, :] = img
        y_true[i] = int(class_names.index(os.path.relpath(dirpath, img_dir)))
        i = i + 1

y_pred = model.predict_classes(batch_holder)

# on réalise des matrices de classification et de confusion
classification, confusion = reports(y_pred, y_true, class_names)

print(classification)
print(confusion)

# on s'assure que le dossier result va contenir les classes résultats
for dirpath, dirnames, filenames in os.walk(img_dir):
    structure = os.path.join(resu_dir, os.path.relpath(dirpath, img_dir))
    if not os.path.isdir(structure):
        os.mkdir(structure)
    else:
        print("Folder already exists")

# on insère les images traitées dans les bonnes classes dans le répertoire result
i = 0
for dirpath, dirnames, filenames in os.walk(img_dir):
    structure = os.path.join(resu_dir, os.path.relpath(dirpath, img_dir))
    for imgnm in filenames:
        shutil.copy(dirpath + '/' + imgnm, resu_dir + '/' + class_names[y_pred[i]] + '/' + imgnm)
        i = i + 1

ValueError: Error when checking input: expected conv2d_1_input to have shape (150, 150, 3) but got array with shape (64, 64, 3)

# Résultats

Par manque de temps, ce notebook n'est pas terminé. Cependant malgré les erreurs rencontrées dans le code, nous pouvons déduire certaines choses. <br>
La méthode New School utilise un réseau de neurones pour indexer des images. 
Premièrement, en donnant des images en entrée au réseau de neurones nous entrainons le modèle qui
servira aux prédictions. Le modèle entrainé permet ensuite de prédire la classe
d’une image.
Cependant, comme nous avons pu le voir, le réseau de neurones est compliqué à mettre en place. Il faut comprendre beaucoup de mécanismes non visibles dans le code seul. Il faut ensuite gérer la limite entre le sur-apprentissage et le sous-apprentissage. Une fois cette étape achevé, il est évident que le réseau de neurones devient une méthode fiable d'indexation.